In [ ]:
#LIBRERIAS
import pandas as pd
import os
import boto3
from botocore.exceptions import ClientError
from datetime import datetime

# CONFIGURACIÓN GENERAL
user = "almartmi"
folder_path ="W:/Shared With Me/GFP Documents/Teams and Programs/EU_VPE_Team/02_Change_Management/2024/01 - Projects/01 - Field Issues/00 - Tickets/Dashboard Updates/Dashboard/Raw_Data/"
bucket_name = "voiceofvehicle"
s3_upload_path = "dashboardvcm/"
current_date = datetime.now()
week_number = current_date.isocalendar()[1]
date_datetime = current_date.date()

# FUNCIONES ACL PARA CONSERVAR CONEXIÓN QUICKsight
def preserve_acl_before_upload(s3, bucket, key):
    try:
        return s3.get_object_acl(Bucket=bucket, Key=key)
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchKey':
            return None
        else:
            raise

def reapply_acl_after_upload(s3, bucket, key, acl):
    if acl:
        s3.put_object_acl(
            Bucket=bucket,
            Key=key,
            AccessControlPolicy={
                'Grants': acl['Grants'],
                'Owner': acl['Owner']
            }
        )

def upload_to_s3_with_acl(local_path, s3_path, bucket):
    s3 = boto3.client('s3')
    acl = preserve_acl_before_upload(s3, bucket, s3_path)
    s3.upload_file(local_path, bucket, s3_path)
    reapply_acl_after_upload(s3, bucket, s3_path, acl)

# CARGA DE ARCHIVOS .xls
dfs = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xls"):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        df['SnapshotDate'] = pd.to_datetime(file_name.split('.')[0], format='%m_%d_%Y')
        df['Week'] = df['SnapshotDate'].dt.isocalendar().week
        dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

# LIMPIEZA Y RENOMBRADO
rename_map = {
    "Comment": "0 Comment",
    "Problem Statement - Anecdote": "1 Problem Statement - Anecdote",
    "Immediate Action": "2 Immediate Action",
    "Problem Definition and Severity": "3 Problem Definition and Severity",
    "Short Term Fix": "4 Short Term Fix",
    "Root Cause": "5 Root Cause",
    "Long term fix technical development and validation": "6 Long term fix technical development and validation",
    "Long term fix deployment": "7 Long term fix deployment",
    "Lessons Learned": "8 Lessons Learned"
}
merged_df['NextStepAction'] = merged_df['NextStepAction'].map(rename_map)
merged_df['CreateDate'] = pd.to_datetime(merged_df['CreateDate'])
merged_df['LastUpdatedDate'] = pd.to_datetime(merged_df['LastUpdatedDate'])

# CALCULOS TEMPORALES
merged_df = merged_df.sort_values(by=['ShortId', 'LastUpdatedDate'])
merged_df['Age'] = (date_datetime - merged_df['CreateDate']).dt.days
merged_df['TimeInSameStatus'] = 0
merged_df['IsLastStatus'] = False

for short_id, group in merged_df.groupby('ShortId'):
    last_index = group.index[-1]
    merged_df.at[last_index, 'IsLastStatus'] = True

    list_date = []
    list_status = []
    for step in group['NextStepAction'].unique():
        sub_group = group[group['NextStepAction'] == step]
        min_date_status = sub_group['LastUpdatedDate'].min()
        list_date.append(min_date_status)
        list_status.append(step)

    for i in range(1, len(list_date)):
        diff = (list_date[i] - list_date[i-1]).days
        merged_df.loc[
            (merged_df['ShortId'] == short_id) & (merged_df['NextStepAction'] == list_status[i-1]),
            'TimeInSameStatus'
        ] = diff

    if list_date:
        final_diff = (date_datetime - list_date[-1]).days
        merged_df.loc[
            (merged_df['ShortId'] == short_id) & (merged_df['NextStepAction'] == list_status[-1]),
            'TimeInSameStatus'
        ] = final_diff

merged_df['TimeInSameStatus'] = merged_df['TimeInSameStatus'].fillna(0).abs()

# CÁLCULO DE MÉTRICAS ADICIONALES
# -----------------------------------------------
# Calcuations of Running totals
merged_df['TotalDaysOpen'] = merged_df.groupby('ShortId')['TimeInSameStatus'].transform('sum')

# Calculation of Goal Metric (ej. 90d para Short Term Fix)
merged_df['FixType'] = merged_df['NextStepAction'].apply(lambda x: 'InShortTermFix' if '4' in str(x) else ('InLongTermFix' if '6' in str(x) else 'Other'))
merged_df['OverTarget'] = merged_df.apply(
    lambda row: row['TotalDaysOpen'] > 90 if row['FixType'] in ['InShortTermFix', 'InLongTermFix'] else False,
    axis=1
)

# EXPORTACIÓN Y SUBIDA
output_filename = f"Data_Merged_with_Calculations_Week_{week_number}.csv"
local_path = os.path.join(folder_path, "Merged_Data", output_filename)
s3_path = f"{s3_upload_path}{output_filename}"

os.makedirs(os.path.dirname(local_path), exist_ok=True)
merged_df.to_csv(local_path, index=False)
upload_to_s3_with_acl(local_path, s3_path, bucket_name)
